<a href="https://colab.research.google.com/github/JerryInterestelar/Collab_notebooks/blob/main/Question%C3%A1ro_Respostas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Programa cientista chefe

# Questão 01

*Com relação à distribuição dos incidentes no tempo, é possível encontrar picos ou linhas de tendência dentro dos dias, meses, semanas ou dos anos?*

Para tal, declararemos as ferramentas basicas à serem usadas. Seram elas as bibliotecas **Pandas** e **plotly**.

In [ ]:
import pandas as pd
import plotly.express as px


Agora usaremos o **Pandas** para "ler" a base de dados, pondo-a dentro de um Dataframe



In [ ]:
data = pd.read_csv("Use_Of_Force.csv")

Com base na analise da base de dados, vemos que é possivel fragmentar a coluna "Occured_date_time" em partes menores: "Date", "hour", "Period". Tais divisoes são necessarias, pois para esse caso em especifico, precisaremos apenas da data "Date".




In [ ]:
data[["Date", "hour", "Period"]] = data["Occured_date_time"].str.split(" ", expand=True)
data["Date"] = pd.to_datetime(data["Date"])

Agora usaremos a função **groupby()** para agrupar as colunas em relação a data desejada, a variavel "frequecias" determina se queremos analisar em relação ao dia, mes, semana e ano.

In [ ]:
frequencias = ["D", "M", "W", "12M"]
data_group = data.groupby(by=[pd.Grouper(key="Date", freq=frequencias[3]), "Incident_Type"])["Incident_Num"]
data_group = data_group.sum().reset_index()


Com isso, para cada tipo de Incidente, apresentado por "Incident_Type", representaremos um grafico Data x Nº de Incidentes juntamente com a media geral para Nº de Incidentes dado.

In [ ]:

types = data["Incident_Type"].unique().tolist()
for typ in types:
    data_type = data_group.loc[data_group["Incident_Type"] == typ].copy()
    data_type["Average_Incident"] = round(data_type["Incident_Num"].mean(), 2)
    type_graph = px.line(data_type, x="Date", y=["Incident_Num", "Average_Incident"], title=f"Linha de tendencia para {typ} ")
    type_graph.show()


Uma vez com os graficos ja calculados, podemos analisar a tendencia do Nº de Incidentes em relação ao tempo para cada tipo de nivel de uso de forca.

# Questão 02

*Uma métrica interessante para a polícia é o grau de reincidência por parte dos civis. Apresente o percentual de casos reincidentes em relação ao total de incidentes em cada setor e verifique se há correlação entre esta métrica e o percentual de incidentes “Level 2” calculado na questão. Que interpretação pode ser dada a este resultado?*

Para isso, dividiremos as operações por setores e para cada uma dela faremos as seguintes operações:


*   Calcular a razão entre a reincidência e o número de Incidentes
*   Calcular a razão entre incidentes do tipo 2 e o número de Incidentes

In [ ]:
SECTORS = data["Sector"].unique().tolist()

for sector in SECTORS:
    if str(sector) == "nan":
        continue
    data_sector = data.loc[data["Sector"] == sector].copy()
    subjects = data_sector["Subject_ID"]
    incidents = data_sector["Incident_Num"]

    l2_data = data_sector.loc[data["Incident_Type"] == "Level 2 - Use of Force"].copy()
    l2_incidents = l2_data["Incident_Num"]

    #Reincidencia / Incidents Num
    ratio_RI = subjects.duplicated().sum() /incidents.sum()

    #Type 2 Incidents / Incidents Num
    ratio_2I = l2_incidents.sum()/incidents.sum()

    print(f"O setor {sector} mostra um percentual de {ratio_RI * 100}% e {ratio_2I * 100}%")

O setor CHARLIE mostra um percentual de 0.0010746444084015856% e 31.95550062231324%
O setor GEORGE mostra um percentual de 0.0017576454321399139% e 24.57722709358947%
O setor KING mostra um percentual de 0.0014380267488546278% e 34.08740967737962%
O setor EDWARD mostra um percentual de 0.0014632232794210638% e 60.866212323825074%
O setor ROBERT mostra um percentual de 0.0013521182427403276% e 19.372179430424524%
O setor SAM mostra um percentual de 0.0014898509827344893% e 21.756281203278146%
O setor LINCOLN mostra um percentual de 0.0016236485209642347% e 21.30910273540326%
O setor UNION mostra um percentual de 0.0017407019145854223% e 19.130157630397115%
O setor DAVID mostra um percentual de 0.001442306964839164% e 27.633165578223924%
O setor FRANK mostra um percentual de 0.001602602828367446% e 20.0627582623825%
O setor JOHN mostra um percentual de 0.001576123042928498% e 11.987078866410021%
O setor NORA mostra um percentual de 0.0014441841657136447% e 17.517669279314426%
O setor BOY

Com tanto os casos reincidentes em relação ao total de incidentes (**ratio_RI**) como os incidentes tipo 2 em relação ao total de incidentes (**ratio_2I**) calculados separadamente para cada seção, podemos agora trabalhar facilmente com base nessas metricas.

# Questão 03

*Considerando que é interessante para a Polícia de Seattle predizer o possível nível do uso da forca policial empregado em uma ocorrência, crie um modelo de classificação utilizando técnicas clássicas de Aprendizagem de Máquina (por exemplo: Regressão logística, Árvores de Decisão, Máquina de Vetores de Suporte (SVM)). Utilize diferentes métricas para avaliar o seu modelo. Qual acurácia seu modelo obteve e a que se deve isto?*

Começaremos com a separação dos atributos da nossa base de dados, onde decidiremos o que vai ser considerado a **entrada** e o **alvo**.

In [ ]:
inputs = data.drop(['ID','Incident_Num', 'Incident_Type', 'Occured_date_time','Date', 'hour', 'Period'], axis='columns')
target = data['Incident_Type']

Para esse modelo, estes serão os atributos que utilizaremos para validar a saida, onde o resultado será um "**Incident_Type**" especifico.

Com a biblioteca **scikit-learn**, utilizaremos a função **fit_transform()** da classe **LabelEnconder** para tranformar toda a nossa base de dados em um formato numérico, visto que esse é formato de dados que o nosso algoritimo suporta.

In [ ]:
from sklearn.preprocessing import LabelEncoder

precinct_l = LabelEncoder()
sector_l = LabelEncoder()
beat_l = LabelEncoder()
officer_l = LabelEncoder()
subject_l = LabelEncoder()
race_l = LabelEncoder()
gender_l = LabelEncoder()

types_l = LabelEncoder()

In [ ]:
inputs['Precinct_n'] = precinct_l.fit_transform(inputs['Precinct'])
inputs['Sector_n'] = sector_l.fit_transform(inputs['Sector'])
inputs['Beat_n'] = beat_l.fit_transform(inputs['Beat'])
inputs['Officer_ID_n'] = officer_l.fit_transform(inputs['Officer_ID'])
inputs['Subject_ID_n'] = subject_l.fit_transform(inputs['Subject_ID'])
inputs['Subject_Race_n'] = race_l.fit_transform(inputs['Subject_Race'])
inputs['Subject_Gender_n'] = gender_l.fit_transform(inputs['Subject_Gender'])

In [ ]:
inputs_n = inputs.drop(['Precinct', 'Sector', 'Beat', 'Officer_ID', 'Subject_ID', 'Subject_Gender', 'Subject_Race', 'Subject_Gender'], axis='columns')
target_n = pd.DataFrame(types_l.fit_transform(target), columns=["Incident_Type_n"])

In [ ]:
inputs_n.head()

In [ ]:
target_n.head()

Para o modelo de classificação, usaremos uma Árvore de decisão do **scikit-learn** que será treinada com "inputs_n" como parâmetros de entrada e "target_n" como parâmetros de saida.

In [ ]:
from sklearn import tree

In [ ]:
model = tree.DecisionTreeClassifier()

In [ ]:
model.fit(inputs_n, target_n)

Com o modelo agora treinado, podemos, com base em parametros especificos, predizer qual o nivel de força preferível à ser aplicado levando em consideração os dados da nossa base de dados.

In [ ]:
model.predict([[7,7,26,81,34,2,2]])